## LOAD vs. INSERT

Let us compare and contrast LOAD and INSERT commands. These are the main approaches using which we get data into Spark Metastore tables.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - DML and Partitioning").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@3fc8800d


org.apache.spark.sql.SparkSession@3fc8800d

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* LOAD will copy the files by dividing them into blocks.
* LOAD is the fastest way of getting data into Spark Metastore tables. However, there will be minimal validations at File level. 
* There will be no transformations or validations at data level.
* If it require any transformation while getting data into Spark Metastore table, then we need to use INSERT command.
* Here are some of the usage scenarios of insert:
  * Changing delimiters in case of text file format
  * Changing file format
  * Loading data into partitioned or bucketed tables (if bucketing is supported).
  * Apply any other transformations at data level (widely used)

In [3]:
%%sql

USE itv002461_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

DROP TABLE IF EXISTS order_items

++
||
++
++



In [5]:
%%sql

CREATE TABLE order_items (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) STORED AS parquet

++
||
++
++



In [6]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items'
    INTO TABLE order_items

++
||
++
++



In [7]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [8]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/order_items" !

Found 1 items
-rwxr-xr-x   3 itv002461 supergroup    5408880 2022-05-30 00:24 /user/itv002461/warehouse/itv002461_retail.db/order_items/part-00000


0

In [9]:
%%sql

SELECT * FROM order_items LIMIT 10

	at org.apache.parquet.hadoop.Parq...


Magic sql failed to execute with error: 
Job aborted due to stage failure: Task 1 in stage 0.0 failed 4 times, most recent failure: Lost task 1.3 in stage 0.0 (TID 4, w01.itversity.com, executor 1): java.lang.RuntimeException: hdfs://m01.itversity.com:9000/user/itv002461/warehouse/itv002461_retail.db/order_items/part-00000 is not a Parquet file. expected magic number at tail [80, 65, 82, 49] but found [48, 46, 48, 10]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:524)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:505)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:499)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:448)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$buildReaderWithPartitionValues$1.footerFileMetaData$lzycompute$1(ParquetFileFormat.scala:371)
	at org.apache.spark.sql.execution.datasources.parquet